In [ ]:
!pip install opencv-python tensorflow matplotlib --quiet
import os, cv2, numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import Xception

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DATASET_PATH_NORMAL = '/content/drive/MyDrive/Premonitor/datasets/aau_vap_trimodal/IR_normal/'
DATASET_PATH_HOTSPOT = '/content/drive/MyDrive/Premonitor/datasets/aau_vap_trimodal/IR_hotspot/'


In [ ]:
def load_images(folder, label, img_size=(128,128)):
    images, labels = [], []
    for file in sorted(os.listdir(folder)):
        if file.endswith('.png'):
            img = cv2.imread(os.path.join(folder, file), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, img_size)
            img = img.astype('float32') / 255.0
            images.append(np.expand_dims(img, -1))
            labels.append(label)
    return np.array(images), np.array(labels)

X1, y1 = load_images(DATASET_PATH_NORMAL, 0)
X2, y2 = load_images(DATASET_PATH_HOTSPOT, 1)

X = np.concatenate([X1, X2], axis=0)
y = np.concatenate([y1, y2], axis=0)
X = X.reshape(-1, 128, 128, 1)


In [ ]:
def build_siamese_encoder(input_shape=(128,128,1)):
    base = Xception(include_top=False, weights=None, input_shape=(128,128,3), pooling='avg')
    inputs = layers.Input(shape=input_shape)
    x = layers.Concatenate()([inputs, inputs, inputs])  # Replicate 1-channel to 3
    features = base(x)
    return models.Model(inputs, features)

def build_classifier(encoder):
    inputs = layers.Input(shape=(128,128,1))
    features = encoder(inputs)
    x = layers.Dense(128, activation='relu')(features)
    x = layers.Dense(1, activation='sigmoid')(x)
    return models.Model(inputs, x)

encoder = build_siamese_encoder()
model = build_classifier(encoder)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
model.fit(X, y, validation_split=0.2, batch_size=32, epochs=10)


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name='block14_sepconv2_act'):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]
    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Visualize
sample = np.expand_dims(X[0], axis=0)
heatmap = make_gradcam_heatmap(sample, model)
plt.imshow(np.squeeze(sample), cmap='gray')
plt.imshow(heatmap, cmap='jet', alpha=0.5)
plt.title("Thermal GradCAM")
plt.axis('off')
plt.show()


In [ ]:
model.save('/content/drive/MyDrive/Premonitor/models/thermal_simsiam_fire_detector.h5')